In [138]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import nltk.data
from nltk.tokenize import word_tokenize
import time

In [57]:
model = KeyedVectors.load_word2vec_format("glove_wv_twitter_200d.txt", binary=False)  

In [58]:
model.init_sims(replace=True)

In [5]:
model.save('27B_100d_twitter_w2v_model')

In [8]:
model.save_word2vec_format('27B_100d_twitter_w2v_model.bin', binary=True)

In [9]:
model = KeyedVectors.load_word2vec_format("27B_100d_twitter_w2v_model.bin", binary=True)
model.init_sims(replace=True)

In [12]:
import numpy as np
f = open('happy_active.txt','r')
lines = [line for line in f]
one = np.ones(len(lines),dtype=np.int)
happy_active = pd.DataFrame(data={"id":list(range(len(lines))), "text":lines, "emotion":one})
happy_active.head()

,emotion,id,text
0,1,0,HAPPY ACTIVE\n
1,1,1,RT UID Thank you Lord for bringing me someone ...
2,1,2,#im #happy #again #silly #happy #me # #buzzin ...
3,1,3,"We Can Be Anything We Want To In Life, Literal..."
4,1,4,Cooked spag for eomma. ANGRYICON #happy #birt...


In [13]:
f = open('happy_inactive.txt','r')
lines = [line.strip() for line in f]
two = np.ones(len(lines), dtype=np.int)
two = np.multiply(two,2)
happy_inactive = pd.DataFrame(data={"id":list(range(len(lines))), "text":lines, "emotion":two})
happy_inactive.head()

,emotion,id,text
0,2,0,HAPPY INACTIVE
1,2,1,Feeling obligated to #instagram this cause eve...
2,2,2,Tired little puppy... #chihuahua #papillon #ch...
3,2,3,UID UID bercanda ja' #peace
4,2,4,"Set #peace of mind as your highest goal, and o..."


In [15]:
f = open('unhappy_active.txt','r')
lines = [line.strip() for line in f]
three = np.ones(len(lines),dtype=np.int)
three = np.multiply(three,3)
unhappy_active = pd.DataFrame(data={"id":list(range(len(lines))), "text":lines, "emotion":three})
unhappy_active.head()

,emotion,id,text
0,3,0,UNHAPPY ACTIVE
1,3,1,#nervous #kpopprincess
2,3,2,Finally done with science #stressed
3,3,3,I miss my best friend in the whole wide world....
4,3,4,Interview today #nervous


In [16]:
f = open('unhappy_inactive.txt','r')
lines = [line.strip() for line in f]
four = np.ones(len(lines),dtype=np.int)
four = np.multiply(four,4)
unhappy_inactive = pd.DataFrame(data={"id":list(range(len(lines))), "text":lines, "emotion":four})
unhappy_inactive.head()

,emotion,id,text
0,4,0,#bored #home #saturday #morning #playing #ps4 ...
1,4,1,Feeling very posey tonight ANGRYICON it doesn...
2,4,2,"Fim de semana, porque  que ests a passar to..."
3,4,3,Dis I'd de first sat I've watched xfactor an J...
4,4,4,RT UID My brother just left me now I'm here wi...


In [19]:
frames = [happy_active, happy_inactive, unhappy_active, unhappy_inactive]
train = pd.concat(frames)
train = train.drop(labels=['id'],axis = 1)
train = train.sample(frac = 1)

index = [i for i in range(127374)]
train['id'] = index

train.head()

,emotion,text,id
20814,2,Ahora toca una duchita calentita #relax,0
15298,2,Hoy toca ducha relajante #relax #velitas #nos...,1
18306,2,#Californication HAPPYICON #relax,2
11931,1,Cart compose myself im going on a date n at y...,3
15617,4,What is wrong with me EXCLAMATION #depressed,4


In [59]:
len(model.wv.syn0[0])

200

In [147]:
# train.to_csv( "Twitter_data_set_mixed_emo4.csv", index=False, quoting=3 )
index2word_set = set(model.wv.index2word)

In [148]:
def clean_tweets( text ):
    # clean_1 = BeautifulSoup(text,"lxml").get_text()
    clean_2 = re.sub("[^a-zA-Z]"," ", text).lower().strip()
    clean_3 = word_tokenize(clean_2)
    stop_words = set(stopwords.words("english")) - set(['and','or','not'])
    words = [w for w in clean_3 if not w in stop_words]
    return words



In [149]:
index2word_set = set(model.wv.index2word)
num_features = 200

def makeFeatureVec(words, num_features = 200):
    global index2word_set
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0.00000000001
    for word in words:
        # if the word is in wordset then add to feature vec
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    
    featureVec = np.divide(featureVec,nwords)
    return featureVec

def get_average_feature_vectors( tweets, num_features = 200 ):
    counter = 0
    tweetFeatureVecs = np.zeros((len(tweets),num_features),dtype="float32")
    
    for tweet in tweets:
        tweetFeatureVecs[counter] = makeFeatureVec(tweet, num_features)
        counter += 1
    
    return tweetFeatureVecs


In [150]:
np.seterr(divide='ignore', invalid='ignore')
start = time.time()

cleaned_tweets = []
for text in train['text']:
    cleaned_tweets.append( clean_tweets(text))
    
trainDataVecs = get_average_feature_vectors( cleaned_tweets, num_features )

end = time.time()

elapsed_cleaning = end - start

In [151]:
import time
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 200 )
start = time.time() 
print("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs, train["emotion"] )

end = time.time()
elapsed_training = end - start

Fitting a random forest to labeled training data...


In [88]:
f_test = open('test_1.txt','r')
lines_test = [line.strip() for line in f_test]

test = pd.DataFrame(data = {'text':lines_test})

In [152]:
cleaned_test_tweets = []
for text in test['text']:
    cleaned_test_tweets.append( clean_tweets(text))

In [153]:
testDataVecs = get_average_feature_vectors( cleaned_test_tweets, num_features )

In [154]:
result = forest.predict( testDataVecs )

In [155]:
counter = 0
for item in result:
    if item == 4:
        counter+=1

In [81]:
a = len(testDataVecs)

In [71]:
print("200d twitter trained glove model")
print("random forest n = 200")
print(counter/a*100)

200d twitter trained glove model
random forest n = 200
78.05926786751888


In [82]:
print('without stop words..')
print("200d twitter trained glove model")
print("random forest n = 200")
print(counter/a*100)

without stop words..
200d twitter trained glove model
random forest n = 200
78.03602556653108


In [93]:
print('without stop words..')
print("200d twitter trained glove model")
print("random forest n = 200")
print(counter/a*100)

without stop words..
200d twitter trained glove model
random forest n = 200
74.7007553747821


In [128]:
print('tokenizer.. and no lower ')
print("200d twitter trained glove model")
print("random forest n = 200")
print(counter/a*100)

tokenizer..
200d twitter trained glove model
random forest n = 200
69.99418942475305


In [144]:
print('tokenizer.. and lower ')
print("200d twitter trained glove model")
print("random forest n = 200")
#print("time taken cleaning "+ elapsed_cleaning)
#print("time taken training "+ elapsed_training)
print(counter/a*100)

tokenizer.. and lower 
200d twitter trained glove model
random forest n = 200
78.11737361998838


In [157]:
print('tokenizer.. and lower and without beautiful soap ')
print("200d twitter trained glove model")
print("random forest n = 200")
print("time taken cleaning "+ str(elapsed_cleaning))
print("time taken training "+ str(elapsed_training))
print(counter/a*100)

tokenizer.. and lower and without beautiful sopa 
200d twitter trained glove model
random forest n = 200
time taken cleaning 40.94924211502075
time taken training 387.54068636894226
78.18710052295177
